# Prediction
We will use rolling window to average the stats of the player, and use them as the input for prediction.

In [46]:
import pandas as pd
import time
import os
from datetime import datetime
import numpy as np

In [7]:
df = pd.read_csv('source/summary.csv')

In [5]:
df = df.sort_values(by = ['Unnamed: 0_level_0_Player','Unnamed: 4_level_0_Age'])

In [37]:
dfa = df[['Unnamed: 3_level_0_Pos','Unnamed: 0_level_0_Player']]

In [38]:
dfa['Unnamed: 0_level_0_Player'][0]

'Gabriel Jesus'

In [16]:
positions = df['Unnamed: 3_level_0_Pos'].astype(str)

# Attackers (FW, LW, RW, ST)
attackers = df[positions.str.contains('FW|LW|RW|ST|AM')]

# Load Model

In [47]:
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import requests
from io import BytesIO
import json
import tempfile

# model = xgb.XGBClassifier()
# model.load_model('model/' + "model.json")

In [48]:
response = requests.get('https://raw.githubusercontent.com/WasuWata/fb_predict/main/code/model/model.json')
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as tmp:
            tmp.write(response.text)
            tmp_path = tmp.name
model = xgb.XGBClassifier()
model.load_model(tmp_path)

In [6]:
team = []
df = {'1':2}
team.append(df)
team = pd.concat(team)
team

TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

# Make a prediction
Set that wanna predict Manchester United vs. Liverpool (based on current performance (summary.csv))

In [49]:
manutd_squad = [
    'Bruno Fernandes', 'Diogo Dalot', 'Casemiro', 'Matthijs de Ligt',
    'Bryan Mbeumo', 'Senne Lammens', 'Amad Diallo', 'Matheus Cunha',
    'Mason Mount', 'Harry Maguire', 'Luke Shaw'
] # Need the user to select their own squad
liverpool_squad = [
    'Mohamed Salah', 'Virgil van Dijk', 'Giorgi  Mamardashvili', 'Alexander Isak',
    'Ryan Gravenberch', 'Milos Kerkez', 'Conor Bradley', 'Dominik Szoboszlai',
    'Alexis Mac Allister', 'Cody Gakpo', 'Ibrahima Konaté'
] # Need the user to select their own squad

In [50]:
home_df = []
away_df = []
for player in manutd_squad:
    player_df = df[df['Unnamed: 0_level_0_Player'] == player].iloc[-3:,:]
    player_df_average = player_df.groupby('Unnamed: 0_level_0_Player').mean(numeric_only = True)
    player_df_average['Team_Team'] = player_df['Team_Team'].unique()[-1]
    player_df_average['Unnamed: 3_level_0_Pos'] = player_df['Unnamed: 3_level_0_Pos'].unique()[-1]
    home_df.append(player_df_average)
home_df = pd.concat(home_df)
for player in manutd_squad:
    player_df = df[df['Unnamed: 0_level_0_Player'] == player].iloc[-3:,:]
    player_df_average = player_df.groupby('Unnamed: 0_level_0_Player').mean(numeric_only = True)
    player_df_average['Team_Team'] = player_df['Team_Team'].unique()[-1]
    player_df_average['Unnamed: 3_level_0_Pos'] = player_df['Unnamed: 3_level_0_Pos'].unique()[-1]
    away_df.append(player_df_average)

away_df = pd.concat(away_df)

In [51]:
# Convert to 90-minutes stat
home_df['Performance.4_Sh'] = home_df['Performance.4_Sh'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
home_df['Performance.5_SoT'] = home_df['Performance.5_SoT'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
home_df['SCA_SCA'] = home_df['SCA_SCA'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
home_df['Performance.9_Tkl'] = home_df['Performance.9_Tkl'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
home_df['Performance.10_Int'] = home_df['Performance.10_Int'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
home_df['Performance.11_Blocks'] = home_df['Performance.11_Blocks'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']

away_df['Performance.4_Sh'] = away_df['Performance.4_Sh'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
away_df['Performance.5_SoT'] = away_df['Performance.5_SoT'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
away_df['SCA_SCA'] = away_df['SCA_SCA'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
away_df['Performance.9_Tkl'] = away_df['Performance.9_Tkl'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
away_df['Performance.10_Int'] = away_df['Performance.10_Int'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']
away_df['Performance.11_Blocks'] = away_df['Performance.11_Blocks'].astype('float32')*90/home_df['Unnamed: 5_level_0_Min']

In [11]:
home_df['Unnamed: 5_level_0_Min'] = 90.0
away_df['Unnamed: 5_level_0_Min'] = 90.0

In [12]:
home_df

,Unnamed: 1_level_0_#,Unnamed: 5_level_0_Min,Performance_Gls,Performance.1_Ast,Performance.2_PK,Performance.3_PKatt,Performance.4_Sh,Performance.5_SoT,Performance.6_CrdY,Performance.7_CrdR,...,Passes_Cmp,Passes.1_Att,Passes.2_Cmp%,Passes.3_PrgP,Carries_Carries,Carries.1_PrgC,Take-Ons_Att,Take-Ons.1_Succ,Team_Team,Unnamed: 3_level_0_Pos
Unnamed: 0_level_0_Player,,,,,,,,,,,,,,,,,,,,,
Bruno Fernandes,8.0,90.0,1.000000,0.666667,0.333333,0.333333,3.200000,0.800000,0.000000,0.0,...,41.666667,54.333333,76.866667,7.333333,29.333333,1.666667,1.333333,0.333333,Manchester United,CM
Diogo Dalot,2.0,90.0,0.000000,0.333333,0.000000,0.000000,0.684411,0.684411,0.666667,0.0,...,34.666667,46.333333,75.266667,4.333333,24.000000,1.333333,1.333333,0.666667,Manchester United,"WB,CB"
Casemiro,18.0,90.0,0.333333,0.000000,0.000000,0.000000,3.568282,0.396476,0.333333,0.0,...,34.666667,45.666667,76.333333,5.333333,25.666667,0.666667,0.000000,0.000000,Manchester United,CM
Matthijs de Ligt,4.0,90.0,0.333333,0.000000,0.000000,0.000000,1.666667,0.666667,0.000000,0.0,...,48.333333,54.666667,87.933333,2.333333,37.000000,0.000000,0.000000,0.000000,Manchester United,CB
Bryan Mbeumo,19.0,90.0,0.333333,0.000000,0.000000,0.000000,3.345725,2.007435,0.333333,0.0,...,29.666667,38.000000,77.633333,3.000000,29.666667,2.000000,2.666667,0.666667,Manchester United,AM
Senne Lammens,31.0,90.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,18.666667,27.666667,67.466667,0.000000,20.333333,0.000000,0.000000,0.000000,Manchester United,GK
Amad Diallo,16.0,90.0,0.333333,0.000000,0.000000,0.000000,2.333333,0.333333,0.000000,0.0,...,40.666667,49.000000,82.533333,2.666667,34.000000,7.333333,5.000000,2.666667,Manchester United,WB
Matheus Cunha,10.0,90.0,0.666667,0.333333,0.000000,0.000000,7.333333,2.333333,0.000000,0.0,...,32.666667,40.333333,80.400000,5.000000,36.333333,5.666667,3.666667,1.666667,Manchester United,AM
Mason Mount,7.0,90.0,0.333333,0.000000,0.000000,0.000000,2.614108,0.746888,0.000000,0.0,...,26.000000,33.000000,80.600000,5.000000,26.333333,1.666667,2.000000,0.333333,Manchester United,AM


In [52]:
def extract_team_features(team_df, is_home = True):
    features = {}


    features['avg_minutes'] = team_df['Unnamed: 5_level_0_Min'].astype('float32').mean()
    # Shooting
    features['total_shots'] = team_df['Performance.4_Sh'].astype('float32').sum()
    features['shots_on_target'] = team_df['Performance.5_SoT'].astype('float32').sum()
    features['xG'] = team_df['Expected_xG'].astype('float32').sum()
    features['xAG'] = team_df['Expected.2_xAG'].astype('float32').sum()
    # Passing
    features['key_passes'] = team_df['SCA_SCA'].astype('float32').sum() # Shot creating action
    features['pass_completion'] = team_df['Passes_Cmp'].astype('float32').sum()/team_df['Passes.1_Att'].astype('float32').sum()*100

    # Defensive
    features['tackles'] = team_df['Performance.9_Tkl'].astype('float32').sum()
    features['interception'] = team_df['Performance.10_Int'].astype('float32').sum()
    features['blocks'] = team_df['Performance.11_Blocks'].astype('float32').sum()

    # Cards
    features['yellow_cards'] = team_df['Performance.6_CrdY'].astype('float32').sum()
    features['red_cards'] = team_df['Performance.7_CrdR'].astype('float32').sum()

    # Position-specific
    positions = team_df['Unnamed: 3_level_0_Pos'].astype(str)

    # Attackers (FW, LW, RW, ST)
    attackers = team_df[positions.str.contains('FW|LW|RW|ST|AM')]

    if len(attackers) > 0:
        features['attackers_xG'] = attackers['Expected_xG'].astype('float32').sum()
        features['attackers_shots'] = attackers['Performance.4_Sh'].astype('float32').sum()

    midfielders = team_df[positions.str.contains('CM|DM|LM|RM|AM')]
    
    if len(midfielders) > 0:
        features['midfielders_passes'] = midfielders['Passes_Cmp'].astype('float32').sum()/midfielders['Passes.1_Att'].astype('float32').sum()*100

    defenders = team_df[positions.str.contains('CB|RB|LB|WB|DF')]

    if len(defenders) > 0:
        features['defenders_tackles'] = defenders['Performance.9_Tkl'].astype('float32').sum()
        features['defenders_blocks'] = defenders['Performance.11_Blocks'].astype('float32').sum()

    return features

def process_match_file(home_df, away_df):
    home_data = home_df
    away_data = away_df
    home_features = extract_team_features(home_data, is_home = True)
    away_features = extract_team_features(away_data, is_home = False)

    match_features = {}
    for key, value in home_features.items():
        match_features[f'home_{key}'] = value

    for key, value in away_features.items():
        match_features[f'away_{key}'] = value

    for key in home_features.keys():
        if key in away_features:
            match_features[f'diff_{key}'] = home_features[key] - away_features[key]
            match_features[f'ratio_{key}'] = home_features[key]/(away_features[key] + 0.00001) # avoid dividing by zero

    return match_features 

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [54]:
X = process_match_file(home_df, away_df)

In [73]:
X.keys()

dict_keys(['home_avg_minutes', 'home_total_shots', 'home_shots_on_target', 'home_xG', 'home_xAG', 'home_key_passes', 'home_pass_completion', 'home_tackles', 'home_interception', 'home_blocks', 'home_yellow_cards', 'home_red_cards', 'home_attackers_xG', 'home_attackers_shots', 'home_midfielders_passes', 'home_defenders_tackles', 'home_defenders_blocks', 'away_avg_minutes', 'away_total_shots', 'away_shots_on_target', 'away_xG', 'away_xAG', 'away_key_passes', 'away_pass_completion', 'away_tackles', 'away_interception', 'away_blocks', 'away_yellow_cards', 'away_red_cards', 'away_attackers_xG', 'away_attackers_shots', 'away_midfielders_passes', 'away_defenders_tackles', 'away_defenders_blocks', 'diff_avg_minutes', 'ratio_avg_minutes', 'diff_total_shots', 'ratio_total_shots', 'diff_shots_on_target', 'ratio_shots_on_target', 'diff_xG', 'ratio_xG', 'diff_xAG', 'ratio_xAG', 'diff_key_passes', 'ratio_key_passes', 'diff_pass_completion', 'ratio_pass_completion', 'diff_tackles', 'ratio_tackles', '

In [55]:
import joblib
# match_df = pd.DataFrame([X])
scaler = joblib.load('scaler/scaler.pkl')
scaler

StandardScaler()

In [56]:
scaler_url = 'https://raw.githubusercontent.com/WasuWata/fb_predict/main/code/scaler/scaler.pkl'
import requests
from io import BytesIO
response = requests.get(scaler_url)
scaler = joblib.load(BytesIO(response.content))

In [60]:
match_df = pd.DataFrame([X])
match_df_scaled = scaler.transform(match_df)

In [61]:
match_result = model.predict(match_df_scaled)

In [62]:
match_result # 0 = Home Win, 1 = Draw, 2 = Away Win

array([1], dtype=int64)

In [66]:
prob = model.predict_proba(match_df_scaled)
prob[0][1]

0.9878364

In [ ]:
df[df['Unnamed: 0_level_0_Player']== 'Emiliano Martínez']

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_#,Unnamed: 2_level_0_Nation,Unnamed: 3_level_0_Pos,Unnamed: 4_level_0_Age,Unnamed: 5_level_0_Min,Performance_Gls,Performance.1_Ast,Performance.2_PK,Performance.3_PKatt,...,SCA.1_GCA,Passes_Cmp,Passes.1_Att,Passes.2_Cmp%,Passes.3_PrgP,Carries_Carries,Carries.1_PrgC,Take-Ons_Att,Take-Ons.1_Succ,Team_Team
2483,Victor Lindelöf,2.0,se SWE,CB,27-280,90,0,0,0,0,...,0,49,51,96.1,5,43,2,0,0,Manchester United
2510,Victor Lindelöf,2.0,se SWE,CB,23-138,90,0,0,0,0,...,0,9,11,81.8,0,5,0,0,0,Manchester United
2567,Victor Lindelöf,2.0,se SWE,CB,25-168,90,0,0,0,0,...,0,49,55,89.1,3,31,0,1,1,Manchester United
2618,Victor Lindelöf,2.0,se SWE,CB,26-197,90,0,0,0,0,...,0,62,70,88.6,2,39,0,0,0,Manchester United
2644,Victor Lindelöf,2.0,se SWE,CB,24-236,90,0,0,0,0,...,0,43,50,86.0,0,34,0,0,0,Manchester United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84524,Victor Lindelöf,2.0,se SWE,CB,30-102,7,0,0,0,0,...,0,14,16,87.5,4,11,0,0,0,Manchester United
84581,Victor Lindelöf,2.0,se SWE,CB,25-067,90,0,0,0,0,...,0,66,79,83.5,6,46,0,0,0,Manchester United
84606,Victor Lindelöf,2.0,se SWE,CB,24-074,56,0,0,0,0,...,0,25,28,89.3,2,17,1,0,0,Manchester United
88673,Victor Lindelöf,2.0,se SWE,CB,24-259,90,0,0,0,0,...,0,64,70,91.4,4,43,1,0,0,Manchester United
